In [17]:
%load_ext autoreload
%autoreload 2
# this only works on startup!
from jax import config
config.update("jax_enable_x64", True)

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
from encoding_information.gpu_utils import limit_gpu_memory_growth
limit_gpu_memory_growth()

from cleanplots import *
from tqdm import tqdm
from encoding_information.information_estimation import *
from encoding_information.image_utils import *
from encoding_information.models.gaussian_process import StationaryGaussianProcess

from encoding_information.bsccm_utils import *
from bsccm import BSCCM
from jax import jit
import numpy as onp
import jax.numpy as np


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from encoding_information.bsccm_utils import *
from encoding_information.image_utils import *

bsccm = BSCCM('/home/hpinkard_waller/data/BSCCM/')

# load images, extract patches, and compute cov mats
edge_crop = 32
patch_size = 10
num_images = 1000

# channel = 'LED119'
channel = 'DPC_Right'

num_test_images = 500

all_images = load_bsccm_images(bsccm, channel=channel, num_images=num_images + num_test_images, edge_crop=edge_crop, median_filter=False)
images = all_images[:num_images]
test_images = all_images[num_images:]
test_patches = extract_patches(test_images, patch_size, num_patches=num_test_images, seed=0)

num_patches = 1000
seed = 10

patches = extract_patches(images, patch_size, num_patches=num_patches, seed=seed)


Opening BSCCM
Opened BSCCM


In [27]:
eigenvalue_floor = 1e-3

from encoding_information.models.pixel_cnn import PixelCNN

pixel_cnn = PixelCNN()

pixel_cnn.fit(patches, max_epochs=5, num_val_samples=100)

Computing loss: 2it [00:02,  1.49s/it]


Initial validation NLL: 369.88


Epoch 1: 100%|██████████| 100/100 [00:07<00:00, 13.60it/s]
Computing loss: 2it [00:00, 24.61it/s]


Epoch 1: validation NLL: 5.96


Epoch 2: 100%|██████████| 100/100 [00:01<00:00, 70.51it/s]
Computing loss: 2it [00:00, 62.87it/s]


...

Epoch 4: 100%|██████████| 100/100 [00:01<00:00, 69.69it/s]
Computing loss: 2it [00:00, 68.83it/s]


Epoch 4: validation NLL: 5.20


Epoch 5: 100%|██████████| 100/100 [00:01<00:00, 70.35it/s]
Computing loss: 2it [00:00, 63.63it/s]

Epoch 5: validation NLL: 5.03


[369.8825988769531,
 5.964324474334717,
 5.471066951751709,
 5.205574035644531,
 5.197755813598633,
 5.030706405639648]

In [1]:
samples = pixel_cnn.generate_samples(6, 20)

fig, axs = plt.subplots(1, 6, figsize=(10, 10))
for k in range(6):
    axs[k].imshow(samples[k], cmap='inferno')
    axs[k].axis('off')
    

NameError: name 'plt' is not defined